In [767]:
from copy import deepcopy
from typing import Tuple, List
from IPython.display import display
from ipywidgets import HTML
from enum import Enum
from itertools import product
from math import sqrt, inf
import random

In [768]:
# Identificadores de las entidades.
Vacio   =   0
Agente  =   1
Wumpus  =   2
Hedor   =   3
Oro     =   4
Brisa   =   5
Hoyo    =   6

# Valor inválido.
Invalido = -1

# Definición de una celda vacía.
Celda_Vacia = [True, False, False, False, False, False, False]

# Sprites para cada una de las entidades del tablero.
Sprites = {
    #      0      1      2      3      4      5      6
    str([True, False, False, False, False, False, False])    :    "vacio",
    str([False, True, False, False, False, False, False])    :    "agente",
    str([False, False, True, False, False, False, False])    :    "wumpus",
    str([False, False, False, True, False, False, False])    :    "hedor",
    str([False, False, False, False, True, False, False])    :    "oro",
    str([False, False, False, False, False, True, False])    :    "brisa",
    str([False, False, False, False, False, False, True])    :    "hoyo",

    str([False, False, False, True, True, False, False])     :    "hedor_oro",
    str([False, False, False, True, False, True, False])     :    "hedor_brisa",
    str([False, False, False, False, True, True, False])     :    "brisa_oro",
    str([False, True, False, True, False, False, False])     :    "agente_hedor",
    str([False, True, False, False, False, True, False])     :    "agente_brisa",
    str([False, True, False, False, True, False, False])     :    "agente_oro",
    str([False, True, True, False, False, False, False])     :    "agente_wumpus",
    str([False, True, False, False, False, False, True])     :    "agente_hoyo",

    str([False, False, False, True, True, True, False])      :    "hedor_oro_brisa",
    str([False, True, False, True, False, True, False])      :    "agente_hedor_brisa",
    str([False, True, False, True, True, False, False])      :    "agente_hedor_oro",
    str([False, True, False, False, True, True, False])      :    "agente_brisa_oro",
    
    str([False, True, False, True, True, True, False])      :    "agente_hedor_oro_brisa"
}

# Distancias mínimas requeridas en la formación inicial del tablero.
Dist_Wumpus_Hoyo = 2
Dist_Wumpus_Agente = 2
Dist_Wumpus_Oro = 1
Dist_Hoyo_Agente = 2
Dist_Hoyo_Oro = 1
Dist_Oro_Agente = 2


# Reglas para respetar las distancias requeridas entre entidades.
# Formato: [ID entidad, Distancia requerida con esa entidad]

# Reglas para el Wumpus.
Reglas_Wumpus = [
    [Agente, Dist_Wumpus_Agente],
    [Hoyo, Dist_Wumpus_Hoyo], 
    [Oro, Dist_Wumpus_Oro]
]

# Reglas para los hoyos.
Reglas_Hoyos = [
    [Agente, Dist_Hoyo_Agente],
    [Wumpus, Dist_Wumpus_Hoyo],
    [Oro, Dist_Hoyo_Oro]
]

# Reglas para el oro.
Reglas_Oro = [
    [Agente, Dist_Oro_Agente],
    [Wumpus, Dist_Wumpus_Oro],
    [Hoyo, Dist_Hoyo_Oro]
]

# Score para la función de evaluación del tablero.
Scr_Vacia = +5         # Puntuación cuando la celda está vacía.
Scr_Brisa = -20        # Puntuación cuando la celda contiene brisa.
Scr_Hedor = -10        # Puntuación cuando la celda contiene hedor.
Scr_Disparo = +15      # Puntuación cuando el disparo de la flecha es certero.
Scr_Nueva = +8         # Puntuación cuando se visita una casilla que no se visitó antes.
Scr_Antigua = -3       # Puntuación cuando se visita una casilla ya visita previamente.
Scr_Cerca = +6         # Puntuación cuando se mejora la distancia anterior.
Scr_Lejos = -3         # Puntuación cuando la distancia es igual o peor a la anterior.
Victoria = +100        # Puntuación por encontrar el oro.
Derrota = -100         # Puntuación por caer donde el Wumpus o un hoyo.



# Enumeración de direcciones.
class Direccion(Enum):
    IZQ = 0             # Dirección izquierda.
    DCHA = 1            # Dirección derecha.
    ARR = 2             # Dirección arriba.
    ABJ = 3             # Dirección abajo.


# Enumeración de orientacion del agente.
class Orientacion(Enum):
    N = (-1, 0)         # Orientación Norte.
    S = (1, 0)          # Orientación Sur.
    E = (0, 1)          # Orientación Este.
    W = (0, -1)         # Orientación Oeste.

In [769]:
class Celda:
    '''
    Clase que genera celdas que funcionan como contenedores
    de entidades que se posicionan en un tablero de dos dimensiones.
    Cada celda se corresponde con una coordenada del tablero.
    '''

    
    
    def __init__(self, entidades: List[bool]):
        '''
        Constructor de celda.
        
        Parámetros
        ----------
        entidades : List[bool]
            Entidades iniciales contenidas 
            en la celda.
            
        Returns
        -------
        void
        '''
        self.establecer_entidades(entidades)
        self.establecer_sprite()


    

    def obtener_entidades(self) -> List[bool]:
        '''
        Obtiene las entidades contenidas en la celda.
        
        Parámetros
        ----------
        void
        
        Returns
        -------
        List[bool] : Lista de booleanos que indican las
                    entidades contenidas en la celda.
        '''
        return self.entidades


    

    def establecer_entidades(self, entidades: List[bool]):
        '''
        Establece las entidades que contiene la celda.
        
        Parámetros
        ----------
        entidades : List[bool]
                Entidades que se van a asignar a la celda.
                
        Returns
        -------
        void
        '''
        self.entidades = entidades[:]

    

    
    def esta_vacia(self):
        '''
        Comprueba si la celda está vacía.
        
        Parámetros
        ----------
        void
        
        Returns
        -------
        void
        '''
        return self.entidades[Vacio] and not any(self.entidades[Agente:])

    


    def contiene_entidad(self, entidad: int) -> bool:
        '''
        Comprueba si una entidad está contenida en la celda.
        
        Parámetros
        ----------
        entidad : int
                Entidad que se quiere comprobar.
                
        Returns
        -------
        bool : Verdadero si la entidad está contenida, 
            Falso en caso contrario.
        '''
        return self.entidades[entidad]

    

    
    def contiene_entidades(self, entidades: List[int]) -> bool:
        '''
        Comprueba si alguna entidad está contenida en la celda.
        
        Parámetros
        ----------
        entidad : List[int]
                Entidad que se quiere comprobar.
                
        Returns
        -------
        bool : Verdadero si la entidad está contenida, 
            Falso en caso contrario.
        '''
        for e in entidades:
            if self.entidades[e]: return True
        return False


    

    def agregar_entidad(self, entidad: int):
        '''
        Agrega una entidad a la celda.
        
        Parámetros
        ----------
        entidad : int
                Identificador de la entidad que
                se agrega.
                
        Returns
        -------
        void
        '''
        self.entidades[Vacio] = False
        
        # Adapta la celda para colocar el Wumpus o el Hoyo.
        if entidad in [Wumpus, Hoyo]:
            self.entidades[Hedor] = False
            self.entidades[Brisa] = False
            self.entidades[entidad] = True
            
        # Adapta la celda para colocar el Hedor o la Brisa.
        elif entidad in [Hedor, Brisa]: 
            if not self.contiene_entidades([Wumpus, Hoyo]):
                self.entidades[entidad] = True
                
        # Agrega el resto de tipos de entidades (Agente u Oro)
        else:
            self.entidades[entidad] = True
        self.establecer_sprite()



    
    def eliminar_entidad(self, entidad: int):
        '''
        Elimina una entidad de la celda.
        
        Parámetros
        ----------
        entidad : int
                Identificador de la entidad que
                se elimina.
                
        Returns
        -------
        void
        '''
        self.entidades[entidad] = False
        if not any(self.entidades): # Si está vacía, se modifica para reflejarlo.
            self.entidades[Vacio] = True
        self.establecer_sprite() # Actualiza el sprite actual.



    
    def establecer_sprite(self):
        '''
        Establece el sprite para la celda según las entidades contenidas.
        
        Parámetros
        ----------
        entidad : int
                Identificador de la entidad que
                se elimina.
                
        Returns
        -------
        void
        '''
        self.sprite_actual = Sprites[str(self.entidades)]



    
    def obtener_sprite(self):
        '''
        Obtiene el sprite actual asignado a la celda.
        
        Parámetros
        ----------
        void
        
        Returns
        -------
        void
        '''
        return self.sprite_actual

    
        

    def esta_disponible(self, es_vecino: bool = False):
        '''
        Comprueba si la celda está disponible para colocar
        una entidad en concreto.
        
        Parámetros
        ----------
        es_vecino : bool
                Indica si la entidad es un vecino (Hedor, Brisa).
                
        Returns
        -------
        void
        '''
        disponible = False
        if es_vecino:
            disponible = self.esta_vacia() or not self.contiene_entidades([self.entidades[Wumpus], self.entidades[Hoyo]])
        else:
            disponible =  self.esta_vacia() or not self.contiene_entidades([self.entidades[Agente], self.entidades[Wumpus], self.entidades[Hoyo]])
        return disponible
    


    
    def solo_agente(self) -> bool:
        '''
        Comprueba si en la celda solo se encuentra
        el agente.
        
        Parámetros
        ----------
        void
        
        Returns
        -------
        bool : Verdadero si solo está el agente,
            False en caso contrario.
        '''
        return not any(self.entidades[:1] + self.entidades[2:])

In [770]:
class TableroWumpus:

    '''
    Genera un tablero para el juego Wumpus.

    Observaciones: Con el fin de poder visualizar las imágentes
                    que representan el estado del tablero, se 
                    recomienda utilizar la versión 6.4.8 de Jupyter
                    Notebook.
    '''
    
    # Variables estáticas.
    Mejor_Dist = inf     # Mejor distancia registrada en la exploración minimax.
    Visitadas = [tuple]  # Registro de celdas ya visitadas por el agente.
    Flechas = 1          # Flechas de las que dispone el agente.

    # Constructor del tablero, solo se pasa el tamaño y el número de hoyos adversarios.
    def __init__(self, longitud: int, num_hoyos: int):
        '''
        Constructor del tablero Wumpus.
        
        Parámetros
        ----------
        longitud : int
            Tamaño de cada dimensión del tablero.
        num_hoyos : int
            Número de hoyos que habrá en el tablero.
            
        Returns
        -------
        void
        '''
        self.longitud = longitud
        self.num_hoyos = num_hoyos
        # Producto cartesiano para obtener todas las posibles coordenadas del tablero.
        self.coords_posibles = list(product(range(self.longitud), range(self.longitud)))
        self.celdas = []
        self.orientacion_agente = Orientacion.N
        self.disparo_efectuado = False
        self.crear_tablero()
        
    
    # Comprueba la igualdad entre tableros.
    def __eq__(self, otro) -> bool:
        '''
        Comprueba si dos tableros (estados) son iguales.
        
        Parámetros
        ----------
        otro : TableroWumpus.
            Tablero con el que se quiere comparar el actual.
            
        Returns
        -------
        bool : Verdadero si son iguales, Falso en caso contrario.
        '''
        return self.celdas == otro.obtener_celdas()


    # Crea el tablero inicial.
    def crear_tablero(self):
        '''
        Inicializa el tablero creando sus celdas
        y colocando las entidades.
        
        Parámetros
        ----------
        void
        
        Returns
        -------
        void
        '''
        # Inicialización del tablero.
        self.celdas = [[Celda(Celda_Vacia) for e in range(self.longitud)] for e in range(self.longitud)]
        # Colocación del agente.
        self.colocar_agente()
        # Colocación del Oro.
        self.colocar_entidad(Oro, Reglas_Oro)
        # Colocación del Wumpus.
        self.colocar_entidad(Wumpus, Reglas_Wumpus, Hedor)
        # Colocación de hoyos.
        for _ in range(self.num_hoyos):
            self.colocar_entidad(Hoyo, Reglas_Hoyos, Brisa)
            
        
    # Obtiene el tablero actual.
    def obtener_tablero(self):
        '''
        Obtiene una copia profunda del tablero.
        
        Parámetros
        ----------
        void
        
        Returns
        -------
        TableroWumpus : copia del tablero.
        '''
        return deepcopy(self)



    
    def obtener_celdas(self):
        '''
        Obtiene una copia profunda de las celdas
        del tablero.
        
        Parámetros
        ----------
        void
        
        Returns
        -------
        List[List[Celda]] : copia profunda de las celdas.
        '''
        return deepcopy(self.celdas)



    
    def esta_dentro(self, fila: int, col: int) -> bool:
        '''
        Comprueba si una posición está dentro del tablero.
        
        Parámetros
        ----------
        fila : int
            Fila de la posición.
        col : int
            Columna de la posición.
            
        Returns
        -------
        bool : Verdadero si son está dentro, Falso en caso contrario.
        '''
        return (0 <= fila < self.longitud and 0 <= col < self.longitud)



    
    def coords_dentro(self, coords: Tuple[int]) -> bool:
        '''
        Comprueba si las coordenadas están dentro
        del tablero.
        
        Parámetros
        ----------
        coords : tuple
            Coordenadas que se quieren comprobar.
            
        Returns
        -------
        bool : Verdadero si está dentro, Falso en caso contrario.
        '''
        return self.esta_dentro(coords[0], coords[1])

    

    
    def obtener_celda(self, fila: int, col: int) -> Celda:
        '''
        Obtiene una celda en una posición dada.
        
        Parámetros
        ----------
        fila : int
            Fila de la posición.
        col : int
            Columna de la posición.
            
        Returns
        -------
        Celda: celda que se encuentra en la posición.
        '''
        if not self.esta_dentro(fila, col): 
            return None
        return self.celdas[fila][col]



    
    def obtener_posicion(self, entidad: int) -> Tuple[int]:
        '''
        Obtiene las coordenadas de una entidad.
        
        Parámetros
        ----------
        entidad : int
            Entidad que se quiere buscar.
            
        Returns
        -------
        Tuple[int] : coordenadas de la entidad.
        '''
        for fila in range(self.longitud):
            for col in range(self.longitud):
                celda = self.obtener_celda(fila, col)
                if celda.contiene_entidad(entidad):
                    return (fila, col)
        return None



    
    def obtener_posiciones(self, entidad: int) -> List[Tuple[int]]:
        '''
        Obtiene las posiciones de un tipo de entidad.
        Permite obtener las posiciones de entidades repetidas.
        
        Parámetros
        ----------
        entidad : int
            Tipo de entidad que se quiere buscar.
            
        Returns
        -------
        List[Tuple[int, int]] : posiciones para cada entidad coincidente.
        '''
        coords = []
        for fila in range(self.longitud):
            for col in range(self.longitud):
                celda = self.obtener_celda(fila, col)
                if celda.contiene_entidad(entidad):
                    coords.append((fila, col))
        return coords



    
    def calcular_distancia(self, coords_A: Tuple[int], coords_B: Tuple[int]) -> float:
        '''
        Calcula la distancia euclídea entre dos coordenadas.
        
        Parámetros
        ----------
        coords_A : Tuple[int]
            Coordenadas del primer punto.
        coords_B : Tuple[int]
            Coordenadas del segundo punto.
            
        Returns
        -------
        float: distancia medida.
        '''
        if (not self.esta_dentro(coords_A[0], coords_A[1]) or 
            not self.esta_dentro(coords_B[0], coords_B[1])): 
            return Invalido
        return sqrt((coords_A[0] - coords_B[0])**2 + (coords_A[1] - coords_B[1])**2)



    
    def suficiente_dist(self, coords_A: Tuple[int], coords_B: Tuple[int], dist_requerida: int) -> bool:
        '''
        Comprueba si la distancia entre dos puntos
        es suficiente según la distancia mínima requerida.
        
        Parámetros
        ----------
        coords_A : Tuple[int]
            Coordenadas del primer punto.
        coords_B : Tuple[int]
            Coordenadas del segundo punto.
            
        Returns
        -------
        bool : distancia medida.
        '''
        distancia = self.calcular_distancia(coords_A, coords_B)
        return distancia != Invalido and distancia >= dist_requerida




    def es_legal(self, destino: Tuple[int], lista_reglas: List[List]) -> bool:
        '''
        Comprueba si la posición en la que se quiere colocar una
        entidad es correcta (legal).
        
        Parámetros
        ----------
        destino : Tuple[int]
            Coordenada donde se quiere colocar.
            
        Returns
        -------
        bool : Verdadero si la posición es correcta, Falso si no.
        '''
        # No es legal si está fuera del tablero.
        if not self.esta_dentro(destino[0], destino[1]): return False

        # No es legal si hay conflicto de entidades.
        celda = self.obtener_celda(destino[0], destino[1])
        if not celda.esta_disponible(): return False

        # Si hay reglas se comprueban.
        if len(lista_reglas) > 0:
            for reglas in lista_reglas:
                entidad = reglas[0]
                dist_requerida = reglas[1]
                coords_otros = self.obtener_posiciones(entidad)
                for coords in coords_otros:
                    if not self.suficiente_dist(destino, coords, dist_requerida): 
                        return False
        return True


    

    def colocar_vecinos(self, fila: int, col: int, entidad_vecino: int):
        '''
        Coloca los vecinos de una posición. Se entiende como celdas vecinas
        aquellas adyacentes que se encuentran en la dirección de los ejes
        vertical y horizontal.
        
        Parámetros
        ----------
        fila : int
            Fila de la posición de referencia.
        col : int
            Columna de la posición de referencia.
        entidad_vecino : int
            Identificador de la entidad vecina que se 
            va a colocar.
            
        Returns
        -------
        void
        '''
        if not self.esta_dentro(fila, col): return
        # Vecino izquierda.
        if self.esta_dentro(fila, col - 1):
            celda = self.obtener_celda(fila, col - 1)
            if celda.esta_disponible(True): celda.agregar_entidad(entidad_vecino)
        # Vecino derecha.
        if self.esta_dentro(fila, col + 1):
            celda = self.obtener_celda(fila, col + 1)
            if celda.esta_disponible(True): celda.agregar_entidad(entidad_vecino)
        # Vecino arriba.
        if self.esta_dentro(fila - 1, col):
            celda = self.obtener_celda(fila - 1, col)
            if celda.esta_disponible(True): celda.agregar_entidad(entidad_vecino)
        # Vecino abajo.
        if self.esta_dentro(fila + 1, col):
            celda = self.obtener_celda(fila + 1, col)
            if celda.esta_disponible(True): celda.agregar_entidad(entidad_vecino)



    
    def borrar_vecinos(self, fila: int, col: int, entidad_vecino: int):
        '''
        Borra todos los vecinos de una posición de referencia.
        
        Parámetros
        ----------
        fila : int
            Fila de la posición de referencia.
        col : int
            Columna de la posición de referencia.
        entidad_vecino : int
            Identificador de la entidad vecina que se 
            va a borrar.
            
        Returns
        -------
        void
        '''
        if not self.esta_dentro(fila, col): return
        # Vecino izquierda.
        if self.esta_dentro(fila, col - 1):
            celda = self.obtener_celda(fila, col - 1)
            celda.eliminar_entidad(entidad_vecino)
        # Vecino derecha.
        if self.esta_dentro(fila, col + 1):
            celda = self.obtener_celda(fila, col + 1)
            celda.eliminar_entidad(entidad_vecino)
        # Vecino arriba.
        if self.esta_dentro(fila - 1, col):
            celda = self.obtener_celda(fila - 1, col)
            celda.eliminar_entidad(entidad_vecino)
        # Vecino abajo.
        if self.esta_dentro(fila + 1, col):
            celda = self.obtener_celda(fila + 1, col)
            celda.eliminar_entidad(entidad_vecino)




    def colocar_agente(self):
        '''
        Coloca el agente en la posición inicial.
        
        Parámetros
        ----------
        void
        
        Returns
        -------
        void
        '''
        coords_agente = (self.longitud - 1, 0)
        celda_agente = self.obtener_celda(coords_agente[0], coords_agente[1])
        celda_agente.agregar_entidad(Agente)
        self.coords_posibles.remove(coords_agente) # La posición ya no está disponible.

    
        
    
    def colocar_entidad(self, entidad: int, lista_reglas: List[List], entidad_vecina=Invalido):
        '''
        Coloca una entidad de forma aleatoria en el tablero respetando las reglas predefinidas
        en las que se indica la distancia mínima requerida con otro tipo de entidades ya colocadas.
        Además, coloca los vecinos de las entidades que posean vecinos (Brisa, Hedor).
        
        Parámetros
        ----------
        entidad : int
            Entidad que se pretende colocar.
        lista_reglas : List[List[int]]
            Listado de las reglas que se deben cumplir al colocar.
        entidad_vecina : int
            Entidad para colocar vecinos.
        
        Returns
        -------
        void
        '''
        if entidad in [Agente, Hedor, Brisa]: return # Entidades que no se pueden colocar manualmente.

        es_legal = False
        coords = tuple()
        coords_exploradas = [tuple]
        
        while not es_legal:
            # Si no quedan coordenadas disponibles o se han explorado todas, no se debe colocar.
            if not self.coords_posibles: 
                print("ERROR: No hay posiciones disponibles.")
                return
            if len(self.coords_posibles) - len(coords_exploradas) < 1: 
                print("ERROR: Se han explorado todas las posiciones disponibles.")
                return
                
            coords = random.choice(self.coords_posibles) # Posición aleatoria.
            es_legal = self.es_legal(coords, lista_reglas) # Comprobación legalidad.
            coords_exploradas.append(coords) # Se convierte en una posición explorada.

        celda = self.obtener_celda(coords[0], coords[1])
        celda.agregar_entidad(entidad)
        self.coords_posibles.remove(coords) # La coordenada ya no está disponible.

        # Se colocan vecinos.
        if entidad_vecina != Invalido:
           self.colocar_vecinos(coords[0], coords[1], entidad_vecina)
        

    
    
    def normalizar_vector(self, vector: Tuple[int]):
        '''
        Normaliza un vector dado.
        
        Parámetros
        ----------
        vector : Tuple[int]
            Vector que se va a normalizar.
        
        Returns
        -------
        Tuple[int] : vector normalizado.
        '''
        # Magnitud del vector
        magnitud = sqrt(vector[0]**2 + vector[1]**2)
        # Evita la división por cero
        if magnitud != 0:
            return (vector[0]/magnitud, vector[1]/magnitud) # Vector normalizado.
        return vector # Se devuelve el mismo vector si era división por cero.


    
    
    def calcular_orientacion(self, origen: Tuple[int], destino: Tuple[int]) -> Tuple[int]:
        '''
        Calcula la orientación según el movimiento realizado.
        
        Parámetros
        ----------
        origen : Tuple[int]
            Coordenadas de origen.
        destino : Tuple[int]
            Coordenadas de destino.
        
        Returns
        -------
        Tuple[int] : orientación calculada.
        '''
        vector = (destino[0] - origen[0], destino[1] - origen[1])
        return self.normalizar_vector(vector)

    
    

    def establecer_orientacion(self, origen: Tuple[int], destino: Tuple[int]):
        '''
        Establece la orientación del agente según el movimiento
        realizado.
        
        Parámetros
        ----------
        origen : Tuple[int]
            Coordenadas de origen.
        destino : Tuple[int]
            Coordenadas de destino.
        
        Returns
        -------
        void
        '''
        orientaciones = [Orientacion.N, Orientacion.E, Orientacion.S, Orientacion.W]
        orientacion = self.calcular_orientacion(origen, destino)
        if orientacion in orientaciones:
            self.orientacion_agente = orientacion
        
         

    
    def girar_agente(self):
        '''
        Gira la orientación del agente en sentido
        antihorario.
        
        Parámetros
        ----------
        void
        
        Returns
        -------
        void
        '''
        orientaciones = [Orientacion.N, Orientacion.W, Orientacion.S, Orientacion.E]
        orientacion_siguiente = (orientaciones.index(self.orientacion_agente) + 1) % len(orientaciones)
        self.orientacion_agente = orientaciones[orientacion_siguiente]
        
        

        
    def evaluar_disparo(self, coords_agente: Tuple[int]):
        '''
        Evalúa la conveniencia de disparar al Wumpus.
        
        Parámetros
        ----------
        coords_agente : Tuple[int]
            Coordenadas del agente.
        
        Returns
        -------
        void
        '''
        celda_agente = self.obtener_celda(coords_agente[0], coords_agente[1])
        
        if TableroWumpus.Flechas == 0 or not celda_agente.contiene_entidad(Hedor): return
        
        coords_izq = (coords_agente[0], coords_agente[1] - 1)
        coords_dcha = (coords_agente[0], coords_agente[1] + 1)
        coords_arr = (coords_agente[0] - 1, coords_agente[1])
        coords_abj = (coords_agente[0] + 1, coords_agente[1])
        
        if [coords_izq, coords_dcha] in TableroWumpus.Visitadas or (not self.coords_dentro(coords_izq) and not self.coords_dentro(coords_dcha)):
            
            if coords_arr in TableroWumpus.Visitadas or not self.coords_dentro(coords_arr):
                while self.orientacion_agente != Orientacion.S:
                    self.girar_agente()
                self.disparar_wumpus()
                
            elif coords_abj in TableroWumpus.Visitadas or not self.coords_dentro(coords_abj): 
                while self.orientacion_agente != Orientacion.N:
                    self.girar_agente()
                self.disparar_wumpus()
                
        elif [coords_arr, coords_abj] in TableroWumpus.Visitadas or (not self.coords_dentro(coords_arr) and not self.coords_dentro(coords_abj)):
            
            if coords_izq in TableroWumpus.Visitadas or not self.coords_dentro(coords_izq):
                while self.orientacion_agente != Orientacion.E:
                    self.girar_agente()
                self.disparar_wumpus()
                
            elif coords_dcha in TableroWumpus.Visitadas or not self.coords_dentro(coords_dcha): 
                while self.orientacion_agente != Orientacion.W:
                    self.girar_agente()
                self.disparar_wumpus()
                


    
    def disparar_wumpus(self):
        '''
        Acción para intentar disparar al Wumpus.
        
        Parámetros
        ----------
        void
        
        Returns
        -------
        void
        '''
        probabilidad_acierto = 0.25
        probabilidad = random.uniform(0.0, 1.0)

        # Disparo certero, en caso contrario se falla el disparo.
        if probabilidad >= probabilidad_acierto:
            print("Disparo acertado.")
            fila, col = self.obtener_posicion(Wumpus)
            celda_wumpus = self.obtener_celda(fila, col)
            self.borrar_vecinos(fila, col, Hedor)
            celda_wumpus.eliminar_entidad(Wumpus)
        else:
            print("Disparo fallido.")
            
        self.disparo_efectuado = True
        TableroWumpus.Flechas -= 1
    
    


    def can_move_up(self, fila: int, col: int) -> bool:
        '''
        Comprueba si se puede mover hacia arriba.
        
        Parámetros
        ----------
        fila : int
            Fila de la posición.
        col : int
            Columna de la posición.
        
        Returns
        -------
        bool : Verdadero si es posible mover hacia arriba,
            Falso en caso contrario.
        '''
        return self.esta_dentro(fila - 1, col)
  
    
    

    def can_move_down(self, fila: int, col: int) -> bool:
        '''
        Comprueba si se puede mover hacia abajo.
        
        Parámetros
        ----------
        fila : int
            Fila de la posición.
        col : int
            Columna de la posición.
        
        Returns
        -------
        bool : Verdadero si es posible mover hacia abajo,
            Falso en caso contrario.
        '''
        return self.esta_dentro(fila + 1, col)


    

    def can_move_left(self, fila: int, col: int) -> bool:
        '''
        Comprueba si se puede mover hacia la izquierda.
        
        Parámetros
        ----------
        fila : int
            Fila de la posición.
        col : int
            Columna de la posición.
        
        Returns
        -------
        bool : Verdadero si es posible mover hacia la izquierda,
            Falso en caso contrario.
        '''
        return self.esta_dentro(fila, col - 1)
 
    


    def can_move_right(self, fila: int, col: int) -> bool:
        '''
        Comprueba si se puede mover hacia la derecha.
        
        Parámetros
        ----------
        fila : int
            Fila de la posición.
        col : int
            Columna de la posición.
        
        Returns
        -------
        bool : Verdadero si es posible mover hacia la derecha,
            Falso en caso contrario.
        '''
        return self.esta_dentro(fila, col + 1)



    
    def get_available_moves_for_max(self, entidad_max: int) -> List[Tuple[int]]:
        '''
        Obtiene los movimientos disponibles para el jugador MAX.
        
        Parámetros
        ----------
        entidad_max : int
            Identificador de la entidad MAX.
        
        Returns
        -------
        List[Tuple[int]] : lista con las coordenadas disponibles para MAX.
        '''
        movimientos = []
        fila, col = self.obtener_posicion(entidad_max)
        
        if self.can_move_up(fila, col):
            movimientos.append((fila - 1, col))
        if self.can_move_down(fila, col):
            movimientos.append((fila + 1, col))
        if self.can_move_left(fila, col):
            movimientos.append((fila, col - 1))
        if self.can_move_right(fila, col):
            movimientos.append((fila, col + 1))
        return movimientos

    


    def get_available_moves_for_min(self, entidad_min: int) -> (List[Tuple[int]], Tuple[int]):
        '''
        Obtiene los movimientos disponibles para el jugador min
        y se devuelve la posición de la entidad escogida (Hoyo).
        
        Parámetros
        ----------
        entidad_min : int
            Identificador de la entidad min.
        
        Returns
        -------
        (List[Tuple[int]], Tuple[int]) : lista con las coordenadas 
                            disponibles para min y sus coordenadas 
                            actuales.
        '''
        movimientos = []
        coords_adversarios = self.obtener_posiciones(entidad_min)
        fila, col = random.choice(coords_adversarios)
        
        if self.can_move_up(fila, col):
            celda = self.obtener_celda(fila - 1, col)
            if not celda.contiene_entidades([Agente, Wumpus, Hoyo, Oro]):
                movimientos.append((fila - 1, col))
        if self.can_move_down(fila, col):
            celda = self.obtener_celda(fila + 1, col)
            if not celda.contiene_entidades([Agente, Wumpus, Hoyo, Oro]):
                movimientos.append((fila + 1, col))
        if self.can_move_left(fila, col):
            celda = self.obtener_celda(fila, col - 1)
            if not celda.contiene_entidades([Agente, Wumpus, Hoyo, Oro]):
                movimientos.append((fila, col - 1))
        if self.can_move_right(fila, col):
            celda = self.obtener_celda(fila, col + 1)
            if not celda.contiene_entidades([Agente, Wumpus, Hoyo, Oro]):
                movimientos.append((fila, col + 1))
        return movimientos, (fila, col) # Movimientos disponibles y las coordenadas del adversario escogido.



    
    def mover_aleatorio(self):
        '''
        Realiza el movimiento aleatorio de un hoyo.
        
        Parámetros
        ----------
        void
        
        Returns
        -------
        void
        '''
        movs_min, coords_min = self.get_available_moves_for_min(Hoyo)
        if len(movs_min) == 0 or coords_min == None : return
        self.mover_hoyo(coords_min, random.choice(movs_min))


    

    def mover_hoyo(self, origen: Tuple[int], destino: Tuple[int]):
        '''
        Mueve un hoyo de una celda a otra.

        Observaciones: no funcionaría con más de dos hoyos.
        
        Parámetros
        ----------
        origen : Tuple[int]
            Coordenadas de origen.
        destino : Tuple[int]
            Coordenadas de destino.
        
        Returns
        -------
        void
        '''
        vecinos = Brisa
        celda_actual = self.obtener_celda(origen[0], origen[1])
        celda_siguiente = self.obtener_celda(destino[0], destino[1])

        # Se elimina el hoyo de la posición anterior.
        celda_actual.eliminar_entidad(Hoyo)
        # Se obtiene la posición del otro hoyo.
        coords_otro = self.obtener_posicion(Hoyo)
        # Se obtiene la posición del Wumpus.
        coords_wumpus = self.obtener_posicion(Wumpus)
        # Se coloca el hoyo en la celda de destino.
        celda_siguiente.agregar_entidad(Hoyo)

        # Gestión de los vecinos del hoyo que se mueve.
        # Se borran los vecinos del hoyo escogido.
        self.borrar_vecinos(origen[0], origen[1], vecinos)
        # Se colocan los vecinos del hoyo escogido en la nueva posición.
        self.colocar_vecinos(destino[0], destino[1], vecinos)
        # Se vuelven a colocar los vecinos del otro hoyo, porque se podría haber borrado alguno.
        if coords_otro:
            self.colocar_vecinos(coords_otro[0], coords_otro[1], vecinos)
        
        # Si el Wumpus sigue vivo, se borra y colocan los vecinos de nuevo
        # porque el movimiento puede haber borrado alguno de sus vecinos.
        if coords_wumpus:
            self.borrar_vecinos(coords_wumpus[0], coords_wumpus[1], Hedor)
            self.colocar_vecinos(coords_wumpus[0], coords_wumpus[1], Hedor)
        

    

    def mover_agente(self, origen: Tuple[int], destino: Tuple[int]):
        '''
        Realiza un movimiento del agente.
        
        Parámetros
        ----------
        origen : Tuple[int]
            Coordenadas de origen.
        destino : Tuple[int]
            Coordenadas de destino.
        
        Returns
        -------
        void
        '''
        celda_actual = self.obtener_celda(origen[0], origen[1])
        celda_siguiente = self.obtener_celda(destino[0], destino[1])
        # Se actualiza la mejor distancia si es necesario.
        TableroWumpus.Mejor_Dist = min(TableroWumpus.Mejor_Dist, self.calcular_distancia(destino, self.obtener_posicion(Oro)))
        # Se agrega la entidad a la nueva posición.
        celda_actual.eliminar_entidad(Agente)
        celda_siguiente.agregar_entidad(Agente)

        
        
    
    def move_can_be_made(self, entidad: int) -> bool:
        '''
        Comprueba si es posible realizar algún movimiento.
        
        Parámetros
        ----------
        entidad : int
            Identificador de la entidad de referencia.
        
        Returns
        -------
        bool : Verdadero si se puede realizar algún movimiento,
            Falso en caso contrario.
        '''
        fila, col = self.obtener_posicion(entidad)
        return any([self.can_move_up(fila, col), 
                    self.can_move_down(fila, col), 
                    self.can_move_left(fila, col), 
                    self.can_move_right(fila, col)])
        
                


    def evaluar_estado(self) -> int:
        '''
        Evalúa el estado actual del tablero
        y muestra el score obtenido.
        
        Parámetros
        ----------
        void
        
        Returns
        -------
        void
        '''
        coords_agente = self.obtener_posicion(Agente)
        coords_oro = self.obtener_posicion(Oro)
        celda_agente = self.obtener_celda(coords_agente[0], coords_agente[1])
        score = 0
        
        print("-------------------------------------------")
        print(f"\nEvaluando posición: {coords_agente}\n")
        
        if celda_agente.contiene_entidad(Oro):
            print(f"VICTORIA: {Victoria}\n")
            return Victoria
            
        if celda_agente.contiene_entidad(Wumpus) or celda_agente.contiene_entidad(Hoyo):
            print(f"DERROTA: {Derrota}\n")
            return Derrota
            
        if celda_agente.solo_agente():
            print(f"VACIA: +{Scr_Vacia}")
            score += Scr_Vacia
            
        if TableroWumpus.Mejor_Dist >= self.calcular_distancia(coords_agente, coords_oro): # Se compara con la mejor distancia registrada.
            print(f"DISTANCIA: +{Scr_Cerca}")
            score += Scr_Cerca
        else:
            print(f"DISTANCIA: {Scr_Lejos}")
            score += Scr_Lejos
            
        if celda_agente.contiene_entidad(Brisa):
            print(f"BRISA: {Scr_Brisa}")
            score += Scr_Brisa
            
        if celda_agente.contiene_entidad(Hedor):
            print(f"HEDOR: {Scr_Hedor}")
            score += Scr_Hedor
            
        if coords_agente not in TableroWumpus.Visitadas:
            print(f"NUEVA VISITA: +{Scr_Nueva}")
            score += Scr_Nueva
        else:
            print(f"ANTIGUA VISITA: {Scr_Antigua}")
            score += Scr_Antigua
            
        if self.disparo_efectuado and TableroWumpus.Flechas == 0 and not self.obtener_posicion(Wumpus):
                print(f"DISPARO CERTERO: +{Scr_Disparo}")
                score += Scr_Disparo
        
        print(f"\nScore: {score}\n")
        return score
        
            


    def is_gameover(self, score_actual) -> bool:
        '''
        Comprueba si la partida ha acabado.
        
        Parámetros
        ----------
        score_actual : int
            Puntuación obtenida.
        
        Returns
        -------
        void
        '''
        return score_actual <= Derrota or score_actual >= Victoria



    
    def obtener_hijos(self) -> List:
        '''
        Obtiene los estados sucesores del estado actual del tablero.
        
        Parámetros
        ----------
        void
        
        Returns
        -------
        List[TableroWumpus] : lista de los tableros sucesores.
        '''
        coords_agente = self.obtener_posicion(Agente)
        movs_max = self.get_available_moves_for_max(Agente)
        hijos = []

        for mov in movs_max:
            nuevo_hijo = self.obtener_tablero()
            nuevo_hijo.mover_agente(coords_agente, mov)
            nuevo_hijo.establecer_orientacion(coords_agente, mov)
            celda_siguiente = self.obtener_celda(mov[0], mov[1])
            if not celda_siguiente.contiene_entidad(Oro):
                nuevo_hijo.evaluar_disparo(mov)
            hijos.append(nuevo_hijo)
            
        return hijos
        
        

    
    def mostrar_tablero(self):
        '''
        Muestra el tablero actual.
        
        Parámetros
        ----------
        void
        
        Returns
        -------
        HTML : cadena html para mostrar el tablero.
        '''
        
        html_string = "<style> img.game {width: 50px !important; height: 50px !important;}</style><table>"
        nueva_fila = "<tr>"
        final_columna = "</tr>"
    
        for i in range(self.longitud):
            
            html_string += nueva_fila
            for j in range(self.longitud):
                celda = self.obtener_celda(i, j)
                sprite = celda.obtener_sprite()
                html = f'<td><img class="game" src="./sprites/{sprite}.png" alt="{sprite}"></img></td>' 
                html_string += html
            html_string += final_columna
            
        html_string += "</table>"
        return HTML(html_string) # Muestra el tablero HTML.

In [771]:
def minimax(estado: TableroWumpus, nivel_actual: int, nivel_max: int, jugador: int, alpha: int, beta: int) -> Tuple[TableroWumpus, int]:
    '''
    Algoritmo MiniMAX que permite obtener el mejor movimiento posible para el jugador.
    
    Parámetros
    ----------
    estado : TableroWumpus
        El estado que se quiere explorar.
    nivel_actual : int
        Nivel de profundidad de exploración.
    nivel_max : int
        Frontera de exploración.
    jugador : int
        Identificador del jugador.
    alpha : int
        Valor alpha.
    beta : int
        Valor beta.
    
    Returns
    -------
    Tuple[TableroWumpus, int] : estado obtenido y su puntuación.
    '''
    
    score = estado.evaluar_estado() # Este score no influye en la decisión, no se puede quedar en la misma posición el agente.
    
    # Comprueba si el jugador ya no puede hacer movimientos o se ha alcanzado la frontera de exploración (¿Es terminal?)
    if (not estado.move_can_be_made(jugador) or nivel_actual == nivel_max):
        return (estado, score) # Fin de la recursión.

    # Hay movimientos disponibles -> hay sucesores.
    hijos = estado.obtener_hijos()

    # Si es el jugador max.
    if jugador == Agente:

        # El mejor estado inicial es el propio estado inicial.
        # Si se considera la posibilidad de no moverse: (max_estado, max_score) = (estado, score)
        (max_estado, max_score) = (None, -inf) 
        
        # Minimax para cada hijo.
        for hijo in hijos:
            (sucesor, score) = minimax(hijo, nivel_actual+1, nivel_max, Hoyo, alpha, beta)

            # Si el score es mejor, se toma como mejor estado.
            if score > max_score:
                (max_estado, max_score) = (sucesor, score)

            # Si al actualizar el mejor score es mayor que beta, se poda, 
            # no hace falta comprobar más hijos, se devuelve el mejor valor actual.
            if max_score >= beta:
                break
                
            # Se toma el mejor valor para alpha.
            alpha = max(alpha, max_score)
                
        return (max_estado, max_score) # Se devuelve el mejor estado conseguido.

    # Si el jugador es min.
    else:
        
        # Inicialización.
        # Si se considera la posibilidad de no moverse: (min_estado, min_score) = (estado, score)
        (min_estado, min_score) = (None, inf)

        # Minimax para cada hijo.
        for hijo in hijos:
            (sucesor, score) = minimax(hijo, nivel_actual+1, nivel_max, Agente, alpha, beta)

            # Si el score es mejor, se toma como mejor estado.
            if score < min_score:
                (min_estado, min_score) = (sucesor, score)

            # Si al actualizar el mejor score es menor o igual que alpha, se poda, 
            # no hace falta comprobar más hijos, se devuelve el mejor valor actual.
            if min_score <= alpha:
                break
                
            # Se toma el mejor valor para beta.
            beta = min(beta, min_score)
                
        return (min_estado, min_score) # Se devuelve el mejor estado conseguido.

In [772]:
# EJECUCIÓN DEL JUEGO.

nivel_inicial = 0 
nivel_maximo = 1 # 1 exploración por turno, solo se exploran celdas adyacentes.
jugador = Agente # Jugador MAX
score = 0 # Puntuación inicial.

tablero = TableroWumpus(4, 2)
print("Tablero inicial:")
display(tablero.mostrar_tablero())

# Mientras no se acabe la partida, se sigue.
while not tablero.is_gameover(score):
    # Se registra la posición del agente como celda visitada.
    TableroWumpus.Visitadas.append(tablero.obtener_posicion(Agente))
    # Ejecución de minimax, se obtiene el mejor movimiento siguiente.
    tablero, score = minimax(tablero, nivel_inicial, nivel_maximo, jugador, -inf, inf)
    # Se reinicia la mejor distancia.
    TableroWumpus.Mejor_Dist = inf
    print("-------------------------------------------")
    print("\nTurno AGENTE")
    print(f"Mejor score: {score}")
    print("Mejor estado:")
    display(tablero.mostrar_tablero())

    # Evita que el hoyo mueva cuando ya el agente ha ganado o perdido.
    if not tablero.is_gameover(score):
        print("Turno HOYO:")
        tablero.mover_aleatorio()
        display(tablero.mostrar_tablero())

print("Partida finalizada.")

if score == Victoria:
    print("¡Victoria! El agente ha sobrevivido y ha encontrado el oro.")
else:
    print("Derrota... El agente no lo ha conseguido.")

Tablero inicial:


HTML(value='<style> img.game {width: 50px !important; height: 50px !important;}</style><table><tr><td><img cla…

-------------------------------------------

Evaluando posición: (3, 0)

VACIA: +5
DISTANCIA: +6
ANTIGUA VISITA: -3

Score: 8

-------------------------------------------

Evaluando posición: (2, 0)

VACIA: +5
DISTANCIA: +6
NUEVA VISITA: +8

Score: 19

-------------------------------------------

Evaluando posición: (3, 1)

VACIA: +5
DISTANCIA: +6
NUEVA VISITA: +8

Score: 19

-------------------------------------------

Turno AGENTE
Mejor score: 19
Mejor estado:


HTML(value='<style> img.game {width: 50px !important; height: 50px !important;}</style><table><tr><td><img cla…

Turno HOYO:


...

Turno HOYO:


HTML(value='<style> img.game {width: 50px !important; height: 50px !important;}</style><table><tr><td><img cla…

-------------------------------------------

Evaluando posición: (0, 2)

VACIA: +5
DISTANCIA: +6
ANTIGUA VISITA: -3

Score: 8

-------------------------------------------

Evaluando posición: (1, 2)

DISTANCIA: -3
BRISA: -20
HEDOR: -10
ANTIGUA VISITA: -3

Score: -36

-------------------------------------------

Evaluando posición: (0, 1)

VACIA: +5
DISTANCIA: -3
ANTIGUA VISITA: -3

Score: -1

-------------------------------------------

Evaluando posición: (0, 3)

VICTORIA: 100

-------------------------------------------

Turno AGENTE
Mejor score: 100
Mejor estado:


HTML(value='<style> img.game {width: 50px !important; height: 50px !important;}</style><table><tr><td><img cla…

Partida finalizada.
¡Victoria! El agente ha sobrevivido y ha encontrado el oro.
